In [29]:
import france_data_management as data
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from tqdm import tqdm
import plotly
import plotly.express as px
import numpy as np
colors = px.colors.qualitative.D3 + plotly.colors.DEFAULT_PLOTLY_COLORS + px.colors.qualitative.Plotly + px.colors.qualitative.Dark24 + px.colors.qualitative.Alphabet


## Data Import

In [30]:
df, df_confirmed, dates, df_new, df_tests, _, df_sursaud = data.import_data()

df_region = df.groupby(['regionName', 'jour', 'regionPopulation']).sum().reset_index()
df_region["hosp_regpop"] = df_region["hosp"] / df_region["regionPopulation"]*1000000 
df_region["rea_regpop"] = df_region["rea"] / df_region["regionPopulation"]*1000000 
df_region["dc_new_regpop"] = df_region["dc_new"] / df_region["regionPopulation"]*1000000 
df_region["dc_new_regpop_rolling7"] = df_region["dc_new_regpop"].rolling(window=7).mean()
df_region["dc_new_rolling7"] = df_region["dc_new"].rolling(window=7).mean()

df_tests_tot = df_tests.groupby(['jour']).sum().reset_index()

df_new_region = df_new.groupby(['regionName', 'jour']).sum().reset_index()
df_france = df.groupby('jour').sum().reset_index()

regions = list(dict.fromkeys(list(df['regionName'].values))) 
lits_reas = pd.read_csv('data/france/lits_rea.csv', sep=",")

10it [00:04,  2.03it/s]                      


## Subplots : régions
Ce script génère 4 graphiques contenant les graphiques de :
- nb d'hospitalisés,
- nb d'hospitalisés par habitant des régions,
- nb de réanimations,
- nb de réanimations par habitant des régions,
et ce pour toutes les régions françaises

In [31]:

for val in ["hosp_regpop", "rea_regpop", "dc_new_regpop_rolling7"]: #
    ni, nj = 5, 4
    i, j = 1, 1

    df_region[val+"_new"] = df_region[val].diff()
    max_value = df_region[val].max()
    
    regions_ordered = df_region[df_region['jour'] == dates[-1]].sort_values(by=[val], ascending=False)["regionName"].values
    regions_ordered = list(dict.fromkeys(list(regions_ordered)))[:]
    
    fig = make_subplots(rows=ni, cols=nj, shared_yaxes=True, subplot_titles=[ "<b>"+ str(r) +"</b>" for r in (regions_ordered[:11] + [""] + regions_ordered[11:14]+[""]+regions_ordered[14:])], vertical_spacing = 0.06, horizontal_spacing = 0.01)
    #&#8681;
    
    sub = "<sub>{}par ordre décroissant des hospitalisations actuelles - guillaumerozier.fr</sub>"
    type_ppl = "hospitalisées"
    
    if "rea" in val:
        type_ppl = "en réanimation"
        
    if "dc" in val:
        type_ppl = "décédées"
        sub = "<sub>par million d'habitants, moyenne mobile sur 7 jours - guillaumerozier.fr</sub>"
        
    df_nonobj = df_region.select_dtypes(exclude=['object'])
    df_nonobj['jour'] = df_region['jour']
    
    vals_quantiles=[]
    for q in range(25, 125, 25):
        vals_quantiles.append(df_nonobj.groupby('jour').quantile(q=q/100).reset_index())
        
    for region in regions_ordered:
        data_r = df_region[df_region["regionName"] == region]
        
        data_r[val + "_new"] = data_r[val].diff()
        ordered_values = df_region.sort_values(by=[val + "_new"], ascending=False)[val + "_new"]
        max_value_diff = ordered_values.quantile(.90)
        
        for data_quant in vals_quantiles:
            fig.add_trace(go.Bar(x=data_quant["jour"], y=data_quant[val], marker=dict(color="grey", opacity=0.1) #rgba(230,230,230,0.5)
                        ),
                      i, j)
        
        fig.add_trace(go.Bar(x=data_r["jour"], y=data_r[val],
                            marker=dict(color = data_r[val + "_new"], coloraxis="coloraxis"), ),
                      i, j)
        
        rangemin = "2020-03-15"
        if "dc" in val:
            rangemin = "2020-03-25"
        fig.update_xaxes(title_text="", range=[rangemin, "2020-05-10"], gridcolor='white', ticks="inside", tickformat='%d/%m', tickangle=0, nticks=10, linewidth=1, linecolor='white', row=i, col=j)
        fig.update_yaxes(title_text="", range=[0, max_value], gridcolor='white', linewidth=1, linecolor='white', row=i, col=j)

        j+=1
        if j == nj+1 or ((i >= 3) & (j == nj)):
            i+=1
            j=1
            
        """if j == nj+1:
            i+=1
            j=1"""

    for i in fig['layout']['annotations']:
        i['font'] = dict(size=15)

    #for annotation in fig['layout']['annotations']: 
            #annotation ['x'] = 0.5
    by_million_title = ""
    by_million_legend = ""
    if "pop" in val:
        by_million_title = "pour 1 million d'habitants, "
        by_million_legend = "pour 1M. d'hab."
    
    

    fig.update_layout(
        barmode="overlay",
        margin=dict(
            l=0,
            r=15,
            b=0,
            t=170,
            pad=0
        ),
        bargap=0,
        paper_bgcolor='#fffdf5',#fcf8ed #faf9ed
        plot_bgcolor='#f5f0e4',#f5f0e4 fcf8ed f0e8d5 
        coloraxis=dict(colorscale=["green", "#ffc832", "#cf0000"], cmin=-max_value_diff, cmax=max_value_diff), 
                    coloraxis_colorbar=dict(
                        title="Solde quotidien de<br>pers. {}<br>{}<br> &#8205; ".format(type_ppl, by_million_legend),
                        thicknessmode="pixels", thickness=15,
                        lenmode="pixels", len=400,
                        yanchor="bottom", y=0.26, xanchor="left", x=0.85,
                        ticks="outside", tickprefix="  ", ticksuffix=" hosp.",
                        nticks=15,
                        tickfont=dict(size=12),
                        titlefont=dict(size=15)),
                      
                    showlegend=False,
    
                     title={
                        'text': ("COVID19 : <b>nombre de personnes {}</b><br>"+sub).format(type_ppl, by_million_title),
                        'y':0.97,
                        'x':0.5,
                        'xref':"paper",
                         'yref':"container",
                        'xanchor': 'center',
                        'yanchor': 'middle'},
                        titlefont = dict(
                        size=35,
                        )
    )

    fig["layout"]["annotations"] += ( dict(
                            x=0.9,
                            y=0.015,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='Source :<br>Santé Publique France',
                            showarrow = False,
                            font=dict(size=12), 
                            opacity=0.5
                        ),
                            dict(
                            x=0.9,
                            y=0.21,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='25 % des données sont<br>comprises dans la courbe<br>grise la plus foncée,<br><br>50 % dans la deuxième,<br><br>75 % dans la troisième,<br><br>100 % dans la plus claire.',
                            showarrow = False,
                            font=dict(size=16), 
                            opacity=1,
                            align='left'
                        ))
    
    name_fig = "subplots_" + val 
    fig.write_image("images/charts/france/{}.png".format(name_fig), scale=2, width=1300, height=1600)

    fig["layout"]["annotations"] += (
                    dict(
                        x=0.5,
                        y=1,
                        xref='paper',
                        yref='paper',
                        xanchor='center',
                        text='Cliquez sur des éléments de légende pour les ajouter/supprimer',
                        showarrow = False
                        ),
                        )
    plotly.offline.plot(fig, filename = 'images/html_exports/france/{}.html'.format(name_fig), auto_open=False)
    print("> " + name_fig)


    #fig.show()

/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



> subplots_hosp_regpop
> subplots_rea_regpop
> subplots_dc_new_regpop_rolling7


## Subplots : départements
Ce script génère 4 graphiques contenant les graphiques de :
- nb d'hospitalisés par habitant des départements,
et ce pour toutes les régions françaises

In [32]:

import numpy as np
for val in ["hosp_deppop"]: #, "hosp", "rea", "rea_pop"
    ni, nj = 12, 9
    i, j = 1, 1

    #df_region[val+"_new"] = df_region[val].diff()
    max_value = df[val].max()
    
    #deps_ordered = df[df['jour'] == dates[-1]].sort_values(by=[val], ascending=False)["departmentName"].values
    #deps_ordered = list(dict.fromkeys(list(deps_ordered)))[:]
    deps_ordered = np.array(list(dict.fromkeys(list(df["departmentName"].values)))[:])
    deps_ordered_nb = np.array(list(dict.fromkeys(list(df["dep"].values)))[:])
    
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "2A", "200")
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "2B", "201")
    
    ind_deps = np.argsort(deps_ordered_nb.astype(int))
    
    deps_ordered_nb = deps_ordered_nb[ind_deps]
    deps_ordered = deps_ordered[ind_deps]
    
    deps_ordered_nb = deps_ordered_nb.astype(str)
    
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "200", "2A")
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "201", "2B")
    
    titles = []
    k=0
    for case in range(1, ni * nj - 1):
        if case in [80, 81, 89, 90, 98, 99]:
            titles += [""] 
        else:
            titles += ["<b>" + deps_ordered_nb[k] + "</b> - " + deps_ordered[k]]
            k+=1

    fig = make_subplots(rows=ni, cols=nj, shared_yaxes=True, subplot_titles= titles, vertical_spacing = 0.025, horizontal_spacing = 0.002)
    #&#8681;
    
    df_nonobj = df.select_dtypes(exclude=['object'])
    df_nonobj['jour'] = df['jour']
    
    vals_quantiles=[]
    for q in range(25, 125, 25):
        vals_quantiles.append(df_nonobj.groupby('jour').quantile(q=q/100).reset_index())
    
    type_ppl = "hospitalisées"
    if "rea" in val:
        type_ppl = "en réanimation"
    max_values_diff=[]
        
    
    for dep in tqdm(deps_ordered):
        data_dep = df[df["departmentName"] == dep]
        
        data_dep[val + "_new"] = data_dep[val].diff()
        ordered_values = data_dep.sort_values(by=[val + "_new"], ascending=False)[val + "_new"]
        max_values_diff += [ordered_values.quantile(.90)]
        
        for data_quant in vals_quantiles:
            fig.add_trace(go.Bar(x=data_quant["jour"], y=data_quant[val], marker=dict(color="grey", opacity=0.1) #rgba(230,230,230,0.5)
                        ),
                      i, j)
        
        
        fig.add_trace(go.Bar(x=data_dep["jour"], y=data_dep[val],
                            marker=dict(color = data_dep[val + "_new"], coloraxis="coloraxis"), ),
                      i, j)
        
        fig.update_xaxes(title_text="", range=["2020-03-15", "2020-05-01"], gridcolor='white', showgrid=False, ticks="inside", tickformat='%d/%m', tickfont=dict(size=7), tickangle=0, nticks=6, linewidth=0, linecolor='white', row=i, col=j)
        fig.update_yaxes(title_text="", range=[0, max_value], gridcolor='white', linewidth=0, linecolor='white', tickfont=dict(size=7), nticks=8, row=i, col=j)

        j+=1
        if j == nj+1 or ((i >= 9) & (j >= nj-1)): 
            i+=1
            j=1
            
        """if j == nj+1:
            i+=1
            j=1"""
    cnt=0
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14, color = str(colors[regions_ordered.index( df[df['departmentName'] == deps_ordered[cnt]]['regionName'].values[0] )]))
        #print(regions_ordered.index( df[df['departmentName'] == deps_ordered[cnt]]['regionName'].values[0] ))
        cnt+=1
        
    #for annotation in fig['layout']['annotations']: 
            #annotation ['x'] = 0.5
    by_million_title = ""
    by_million_legend = ""
    if "pop" in val:
        by_million_title = "pour 100 000 habitants, "
        by_million_legend = "pour 100k hab."
        
    max_value_diff = np.mean(max_values_diff) * 1.7
    fig.update_layout(
        barmode='overlay',
        margin=dict(
            l=0,
            r=15,
            b=0,
            t=160,
            pad=0
        ),
        bargap=0,
        paper_bgcolor='#fffdf5',#fcf8ed #faf9ed
        plot_bgcolor='#f5f0e4',#f5f0e4 fcf8ed f0e8d5
        coloraxis=dict(colorscale=["green", "#ffc832", "#cf0000"], cmin=-max_value_diff, cmax=max_value_diff), 
                    coloraxis_colorbar=dict(
                        title="Solde quotidien de<br>pers. {}<br>{}<br> &#8205; ".format(type_ppl, by_million_legend),
                        thicknessmode="pixels", thickness=15,
                        lenmode="pixels", len=350,
                        yanchor="bottom", y=0.15, xanchor="left", x=0.87,
                        ticks="outside", tickprefix="  ", ticksuffix=" hosp.",
                        nticks=5,
                        tickfont=dict(size=12),
                        titlefont=dict(size=15)),
                      
                    showlegend=False,
    
                     title={
                        'text': "COVID19 : <b>nombre de personnes {}</b>".format(type_ppl, by_million_title),
                        'y':0.98,
                        'x':0.5,
                        'xref':"paper",
                        'yref':"container",
                        'xanchor': 'center',
                        'yanchor': 'middle'},
                        titlefont = dict(
                            size=50
                        )
    )

    fig["layout"]["annotations"] += ( dict(
                            x=0.9,
                            y=0.015,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='Source :<br>Santé Publique France',
                            showarrow = False,
                            font=dict(size=15), 
                            opacity=0.5
                        ),
                            dict(
                            x=0.9,
                            y=0.14,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='25 % des données sont<br>comprises dans la courbe<br>grise la plus foncée,<br><br>50 % dans la deuxième,<br><br>75 % dans la troisième,<br><br>100 % dans la plus claire.',
                            showarrow = False,
                            font=dict(size=16), 
                            opacity=1,
                            align='left'
                        ),
                            dict(
                            x=0.5,
                            y=1.03,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='middle',
                            text='pour 100 000 habitants de chaque département - guillaumerozier.fr',
                            showarrow = False,
                            font=dict(size=30), 
                            opacity=1
                        ),
                                    )
    
    name_fig = "subplots_dep_" + val 
    fig.write_image("images/charts/france/{}.png".format(name_fig), scale=3, width=1600, height=2300)

    fig["layout"]["annotations"] += (
                    dict(
                        x=0.5,
                        y=1,
                        xref='paper',
                        yref='paper',
                        xanchor='center',
                        text='Cliquez sur des éléments de légende pour les ajouter/supprimer',
                        showarrow = False
                        ),
                        )
    plotly.offline.plot(fig, filename = 'images/html_exports/france/{}.html'.format(name_fig), auto_open=False)
    print("> " + name_fig)


    #fig.show()

/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  0%|          | 0/100 [00:00<?, ?it/s]/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

100%|██████████| 100/100 [00:14<00:00,  6.72it/s]


> subplots_dep_hosp_deppop


## Subplots : départements - classé par régions
Idem précédent mais les départements sont rangés dans leurs régions, et les régions classées par ordre décroissant du nb de personnes

In [33]:

import numpy as np
for val in ["hosp_deppop"]: #, "hosp", "rea", "rea_pop"
    ni, nj = 12, 9
    i, j = 1, 1

    #df_region[val+"_new"] = df_region[val].diff()
    max_value = df[val].max()
    
    #deps_ordered = df[df['jour'] == dates[-1]].sort_values(by=[val], ascending=False)["departmentName"].values
    #deps_ordered = list(dict.fromkeys(list(deps_ordered)))[:]
    
    
    titles = []
    k=0
    deps_by_reg=[]
    case=1
    for reg in regions_ordered:
        deps_ordered = df[(df['regionName']==reg) & (df['jour'] == dates[-1])].sort_values(by=val, ascending=False)
        
        deps_by_reg += [[deps_ordered['dep'].values, deps_ordered['departmentName'].values]]
        
        for d in range(len(deps_ordered)):
            if case in [80, 89, 98]:
                titles += ["", ""]
                titles += ["" + deps_ordered['dep'].values[d] + " - " + deps_ordered['departmentName'].values[d] + ""]
                case+=3
            else:
                titles += ["" + deps_ordered['dep'].values[d] + " - " + deps_ordered['departmentName'].values[d] + ""]
                case+=1
        k+=1
        
    """for case in range(1, ni * nj - 1):
        if case in [80, 81, 89, 90, 98, 99]:
            titles += [""] 
        else:
            titles += ["<b>" + deps_ordered_nb[k] + "</b> - " + deps_ordered[k]]
            k+=1"""

    fig = make_subplots(rows=ni, cols=nj, shared_yaxes=True, subplot_titles= titles, vertical_spacing = 0.030, horizontal_spacing = 0.002)
    #&#8681;
    
    df_nonobj = df.select_dtypes(exclude=['object'])
    df_nonobj['jour'] = df['jour']
    
    vals_quantiles=[]
    for q in range(25, 125, 25):
        vals_quantiles.append(df_nonobj.groupby('jour').quantile(q=q/100).reset_index())
    
    type_ppl = "hospitalisées"
    if "rea" in val:
        type_ppl = "en réanimation"
    max_values_diff=[]
        
    
    for reg in tqdm(deps_by_reg):
        for dep in reg[1]:

            data_dep = df[df["departmentName"] == dep]

            data_dep[val + "_new"] = data_dep[val].diff()
            ordered_values = data_dep.sort_values(by=[val + "_new"], ascending=False)[val + "_new"]
            max_values_diff += [ordered_values.quantile(.90)]

            for data_quant in vals_quantiles:
                fig.add_trace(go.Bar(x=data_quant["jour"], y=data_quant[val], marker=dict(color="grey", opacity=0.1) #rgba(230,230,230,0.5)
                            ),
                          i, j)


            fig.add_trace(go.Bar(x=data_dep["jour"], y=data_dep[val],
                                marker=dict(color = data_dep[val + "_new"], coloraxis="coloraxis"), ),
                          i, j)

            fig.update_xaxes(title_text="", range=["2020-03-15", "2020-05-01"], gridcolor='white', showgrid=False, ticks="inside", tickformat='%d/%m', tickfont=dict(size=7), tickangle=0, nticks=6, linewidth=0, linecolor='white', row=i, col=j)
            fig.update_yaxes(title_text="", range=[0, max_value], gridcolor='white', linewidth=0, linecolor='white', tickfont=dict(size=7), nticks=8, row=i, col=j)

            j+=1
            if j == nj+1 or ((i >= 9) & (j >= nj-1)): 
                i+=1
                j=1

    cnt = 0
    reg = 0
    annotations_to_add = ()
    for i in fig['layout']['annotations']:
        if len(i['text']) > 1:
            
            if ((reg==0) & (cnt==0)) or (cnt == len(deps_by_reg[reg][0])):
                if (cnt == len(deps_by_reg[reg][0])):
                    reg +=1
                    cnt = 0
            annotations_to_add += ( dict(         
                            x=i['x'],
                            y=str(float(i['y'])+0.006),
                            xref= 'paper',
                            yref= 'paper',
                            xanchor= 'center',
                            yanchor= 'bottom',
                            align='left',
                            text= "<b>" + regions_ordered[reg] + "</b>",
                            showarrow = False,
                            font=dict(size=11, color = str(colors[reg])), 
                            opacity=1
                        ),)    
            i['font'] = dict(size=11, color = str(colors[reg]))
            i['align'] = 'center'
            cnt+=1
    
        
    #for annotation in fig['layout']['annotations']: 
            #annotation ['x'] = 0.5
    by_million_title = ""
    by_million_legend = ""
    if "pop" in val:
        by_million_title = "pour 100 000 habitants, "
        by_million_legend = "pour 100k hab."
        
    max_value_diff = np.mean(max_values_diff) * 1.7
    fig.update_layout(
        barmode='overlay',
        margin=dict(
            l=0,
            r=15,
            b=0,
            t=160,
            pad=0
        ),
        bargap=0,
        paper_bgcolor='#fffdf5',#fcf8ed #faf9ed
        plot_bgcolor='#f5f0e4',#f5f0e4 fcf8ed f0e8d5
        coloraxis=dict(colorscale=["green", "#ffc832", "#cf0000"], cmin=-max_value_diff, cmax=max_value_diff), 
                    coloraxis_colorbar=dict(
                        title="Solde quotidien de<br>pers. {}<br>{}<br> &#8205; ".format(type_ppl, by_million_legend),
                        thicknessmode="pixels", thickness=15,
                        lenmode="pixels", len=350,
                        yanchor="bottom", y=0.15, xanchor="left", x=0.87,
                        ticks="outside", tickprefix="  ", ticksuffix=" hosp.",
                        nticks=5,
                        tickfont=dict(size=12),
                        titlefont=dict(size=15)),
                      
                    showlegend=False,
    
                     title={
                        'text': "COVID19 : <b>nombre de personnes {}</b>".format(type_ppl, by_million_title),
                        'y':0.98,
                        'x':0.5,
                        'xref':"paper",
                        'yref':"container",
                        'xanchor': 'center',
                        'yanchor': 'middle'},
                        titlefont = dict(
                            size=50
                        )
    )

    fig["layout"]["annotations"] += ( dict(
                            x=0.9,
                            y=0.015,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='Source :<br>Santé Publique France',
                            showarrow = False,
                            font=dict(size=15), 
                            opacity=0.5
                        ),
                            dict(
                            x=0.9,
                            y=0.14,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='25 % des données sont<br>comprises dans la courbe<br>grise la plus foncée,<br><br>50 % dans la deuxième,<br><br>75 % dans la troisième,<br><br>100 % dans la plus claire.',
                            showarrow = False,
                            font=dict(size=16), 
                            opacity=1,
                            align='left'
                        ),
                            dict(
                            x=0.5,
                            y=1.03,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='middle',
                            text='pour 100 000 habitants de chaque département - guillaumerozier.fr',
                            showarrow = False,
                            font=dict(size=30), 
                            opacity=1
                        ),
                                    ) + annotations_to_add
    
    name_fig = "subplots_dep__class-par-reg" + val 
    fig.write_image("images/charts/france/{}.png".format(name_fig), scale=3, width=1600, height=2300)

    fig["layout"]["annotations"] += (
                    dict(
                        x=0.5,
                        y=1,
                        xref='paper',
                        yref='paper',
                        xanchor='center',
                        text='Cliquez sur des éléments de légende pour les ajouter/supprimer',
                        showarrow = False
                        ),
                        )
    plotly.offline.plot(fig, filename = 'images/html_exports/france/{}.html'.format(name_fig), auto_open=False)
    print("> " + name_fig)


    #fig.show()

/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  0%|          | 0/17 [00:00<?, ?it/s]/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

100%|██████████| 17/17 [00:11<00:00,  1.47it/s]


> subplots_dep__class-par-reghosp_deppop


In [34]:
df_sursaud = df_sursaud[df_sursaud["sursaud_cl_age_corona"] == "0"]
df_sursaud["taux_covid"] = df_sursaud["nbre_pass_corona"] / df_sursaud["nbre_pass_tot"]
#df_sursaud

In [35]:

for val in ["hosp_deppop"]: #, "hosp", "rea", "rea_pop"
    ni, nj = 12, 9
    i, j = 1, 1
    
    regions_ordered = df_region[df_region['jour'] == dates[-1]].sort_values(by=[val], ascending=False)["regionName"].values
    regions_ordered = list(dict.fromkeys(list(regions_ordered)))[:]
    
    #df_region[val+"_new"] = df_region[val].diff()
    max_value = df_sursaud["taux_covid"].max()*100
    
    #deps_ordered = df[df['jour'] == dates[-1]].sort_values(by=[val], ascending=False)["departmentName"].values
    #deps_ordered = list(dict.fromkeys(list(deps_ordered)))[:]
    deps_ordered = np.array(list(dict.fromkeys(list(df["departmentName"].values)))[:])
    deps_ordered_nb = np.array(list(dict.fromkeys(list(df["dep"].values)))[:])
    
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "2A", "200")
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "2B", "201")
    
    ind_deps = np.argsort(deps_ordered_nb.astype(int))
    
    deps_ordered_nb = deps_ordered_nb[ind_deps]
    deps_ordered = deps_ordered[ind_deps]
    
    deps_ordered_nb = deps_ordered_nb.astype(str)
    
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "200", "2A")
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "201", "2B")
    
    titles = []
    k=0
    for case in range(1, ni * nj - 1):
        if case in [80, 81, 89, 90, 98, 99]:
            titles += [""] 
        else:
            titles += ["<b>" + deps_ordered_nb[k] + "</b> - " + deps_ordered[k] + " ↓"]
            k+=1

    fig = make_subplots(rows=ni, cols=nj, shared_yaxes=True, subplot_titles= titles, vertical_spacing = 0.025, horizontal_spacing = 0.002)
    #&#8681;
    
    df_nonobj = df.select_dtypes(exclude=['object'])
    df_nonobj['jour'] = df['jour']
    
    vals_quantiles=[]
    for q in range(25, 125, 25):
        vals_quantiles.append(df_nonobj.groupby('jour').quantile(q=q/100).reset_index())
    
    type_ppl = "hospitalisées"
    if "rea" in val:
        type_ppl = "en réanimation"
    max_values_diff=[]
        
    clrs_dep = []
    for dep in tqdm(deps_ordered_nb):
        data_dep = df[df["departmentName"] == dep]
        
        data_dep[val + "_new"] = data_dep[val].diff()
        ordered_values = data_dep.sort_values(by=[val + "_new"], ascending=False)[val + "_new"]
        max_values_diff += [ordered_values.quantile(.90)]
                
        df_sursaud_dep = df_sursaud[df_sursaud["dep"] == dep]
        
        values_y = df_sursaud_dep["taux_covid"]*100
        clrs=[]
        for tx in values_y.values:
            if tx < 6:
                clrs += ["green"]
            elif tx < 10:
                clrs += ["orange"]
            else:
                clrs += ["red"]
                
        mean_last7 = values_y[-7:].mean()
        
        if mean_last7 < 6:
            clrs_dep += ["green"]
        elif mean_last7 < 10:
            clrs_dep += ["orange"]
        else:
            clrs_dep += ["red"]
            
        fig.add_trace(go.Bar(x = df_sursaud_dep["date_de_passage"], y = values_y,
                            marker_color=clrs),
                      i, j)
        
        fig.update_xaxes(title_text="", range=["2020-04-25", "2020-05-12"], gridcolor='white', showgrid=False, ticks="inside", tickformat='%d/%m', tickfont=dict(size=7), tickangle=0, nticks=6, linewidth=0, linecolor='white', row=i, col=j)
        fig.update_yaxes(title_text="", range=[0, 25], gridcolor='white', linewidth=0, linecolor='white', tickfont=dict(size=7), nticks=8, row=i, col=j)

        j+=1
        if j == nj+1 or ((i >= 9) & (j >= nj-1)): 
            i+=1
            j=1

    cnt=0
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14, color = clrs_dep[cnt])
        #print(regions_ordered.index( df[df['departmentName'] == deps_ordered[cnt]]['regionName'].values[0] ))
        cnt+=1
        
    #for annotation in fig['layout']['annotations']: 
            #annotation ['x'] = 0.5
    by_million_title = ""
    by_million_legend = ""
    if "pop" in val:
        by_million_title = "pour 100 000 habitants, "
        by_million_legend = "pour 100k hab."
        
    max_value_diff = np.mean(max_values_diff) * 1.7
    fig.update_layout(
        barmode='overlay',
        margin=dict(
            l=0,
            r=15,
            b=0,
            t=160,
            pad=0
        ),
        bargap=0,
        paper_bgcolor='#fffdf5',#fcf8ed #faf9ed
        plot_bgcolor='#f5f0e4',#f5f0e4 fcf8ed f0e8d5
        coloraxis=dict(colorscale=["green", "#ffc832", "#cf0000"], cmin=-max_value_diff, cmax=max_value_diff), 
                    coloraxis_colorbar=dict(
                        title="Solde quotidien de<br>pers. {}<br>{}<br> &#8205; ".format(type_ppl, by_million_legend),
                        thicknessmode="pixels", thickness=15,
                        lenmode="pixels", len=350,
                        yanchor="bottom", y=0.15, xanchor="left", x=0.87,
                        ticks="outside", tickprefix="  ", ticksuffix=" hosp.",
                        nticks=5,
                        tickfont=dict(size=12),
                        titlefont=dict(size=15)),
                      
                    showlegend=False,
    
                     title={
                        'text': "Indicateur 1 : <b>circulation du virus</b>",
                        'y':0.98,
                        'x':0.5,
                        'xref':"paper",
                        'yref':"container",
                        'xanchor': 'center',
                        'yanchor': 'middle'},
                        titlefont = dict(
                            size=50
                        )
    )

    fig["layout"]["annotations"] += ( 
                        dict(
                            x=0.9,
                            y=0.17,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='Rouge : > 10 %<br><br><br>Orange : 6 à 10%<br><br><br>Vert : < 6%',
                            showarrow = False,
                            font=dict(size=16), 
                            opacity=1,
                            align='left'
                        ),
                        dict(
                            x=0.9,
                            y=0.015,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='Source :<br>Santé Publique France',
                            showarrow = False,
                            font=dict(size=15), 
                            opacity=0.5
                        ),
                            
                            dict(
                            x=0.5,
                            y=1.03,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='middle',
                            text='taux de suspicion Covid aux urgences - guillaumerozier.fr',
                            showarrow = False,
                            font=dict(size=30), 
                            opacity=1
                        ),
                                    )
    
    name_fig = "subplots_deconf_indic1"
    fig.write_image("images/charts/france/{}.png".format(name_fig), scale=3, width=1600, height=2300)

    fig["layout"]["annotations"] += (
                    dict(
                        x=0.5,
                        y=1,
                        xref='paper',
                        yref='paper',
                        xanchor='center',
                        text='Cliquez sur des éléments de légende pour les ajouter/supprimer',
                        showarrow = False
                        ),
                        )
    plotly.offline.plot(fig, filename = 'images/html_exports/france/{}.html'.format(name_fig), auto_open=False)
    print("> " + name_fig)


    #fig.show()

/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

100%|██████████| 100/100 [00:06<00:00, 14.93it/s]


> subplots_deconf_indic1


In [36]:
df_region

,regionName,jour,regionPopulation,sexe,hosp,rea,rad,dc,regionCode,departmentPopulation,...,hosp_new,hosp_new_deppop,hosp_regpop,rea_regpop,dc_new_regpop,dc_new_regpop_rolling7,dc_new_rolling7,hosp_regpop_new,rea_regpop_new,dc_new_regpop_rolling7_new
0,Auvergne-Rhône-Alpes,2020-03-18,8027000,0,172,35,69,7,1008.0,8069287,...,0.0,0.000000,21.427682,4.360284,0.000000,NaN,NaN,NaN,NaN,NaN
1,Auvergne-Rhône-Alpes,2020-03-19,8027000,0,385,79,132,22,1008.0,8069287,...,213.0,15.045638,47.963124,9.841784,1.868693,NaN,NaN,26.535443,5.481500,NaN
2,Auvergne-Rhône-Alpes,2020-03-20,8027000,0,463,87,179,33,1008.0,8069287,...,78.0,10.364657,57.680329,10.838420,1.370375,NaN,NaN,9.717204,0.996636,NaN
3,Auvergne-Rhône-Alpes,2020-03-21,8027000,0,525,88,192,38,1008.0,8069287,...,62.0,7.355447,65.404261,10.963000,0.622898,NaN,NaN,7.723932,0.124580,NaN
4,Auvergne-Rhône-Alpes,2020-03-22,8027000,0,641,106,212,47,1008.0,8069287,...,116.0,12.712213,79.855488,13.205432,1.121216,NaN,NaN,14.451227,2.242432,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,Provence-Alpes-Côte d'Azur,2020-05-04,5059000,0,1473,222,3973,732,558.0,5091003,...,-30.0,-3.210472,291.164262,43.882190,3.755683,2.146105,10.857143,-5.930026,-2.965013,-0.169429
880,Provence-Alpes-Côte d'Azur,2020-05-05,5059000,0,1385,210,4061,746,558.0,5091003,...,-88.0,-5.720614,273.769520,41.510180,2.767345,2.287296,11.571429,-17.394742,-2.372010,0.141191
881,Provence-Alpes-Côte d'Azur,2020-05-06,5059000,0,1359,195,4127,756,558.0,5091003,...,-26.0,-2.767475,268.630164,38.545167,1.976675,2.089628,10.571429,-5.139356,-2.965013,-0.197668
882,Provence-Alpes-Côte d'Azur,2020-05-07,5059000,0,1337,184,4187,764,558.0,5091003,...,-22.0,-4.203727,264.281479,36.370824,1.581340,1.920199,9.714286,-4.348686,-2.174343,-0.169429


In [ ]:

for val in ["hosp_regpop", "rea_regpop", "dc_new_regpop_rolling7"]: #
    ni, nj = 5, 4
    i, j = 1, 1

    df_region[val+"_new"] = df_region[val].diff()
    max_value = df_region[val].max()
    
    regions_ordered = df_region[df_region['jour'] == dates[-1]].sort_values(by=[val], ascending=False)["regionName"].values
    regions_ordered = list(dict.fromkeys(list(regions_ordered)))[:]
    
    fig = make_subplots(rows=ni, cols=nj, shared_yaxes=True, subplot_titles=[ "<b>"+ str(r) +"</b>" for r in (regions_ordered[:11] + [""] + regions_ordered[11:14]+[""]+regions_ordered[14:])], vertical_spacing = 0.06, horizontal_spacing = 0.01)
    #&#8681;
    
    sub = "<sub>{}par ordre décroissant des hospitalisations actuelles - guillaumerozier.fr</sub>"
    type_ppl = "hospitalisées"
    
    if "rea" in val:
        type_ppl = "en réanimation"
        
    if "dc" in val:
        type_ppl = "décédées"
        sub = "<sub>par million d'habitants, moyenne mobile sur 7 jours - guillaumerozier.fr</sub>"
        
    df_nonobj = df_region.select_dtypes(exclude=['object'])
    df_nonobj['jour'] = df_region['jour']
    
    vals_quantiles=[]
    for q in range(25, 125, 25):
        vals_quantiles.append(df_nonobj.groupby('jour').quantile(q=q/100).reset_index())
        
    for region in regions_ordered:
        data_r = df_region[df_region["regionName"] == region]
        
        data_r[val + "_new"] = data_r[val].diff()
        ordered_values = df_region.sort_values(by=[val + "_new"], ascending=False)[val + "_new"]
        max_value_diff = ordered_values.quantile(.90)
        
        for data_quant in vals_quantiles:
            fig.add_trace(go.Bar(x=data_quant["jour"], y=data_quant[val], marker=dict(color="grey", opacity=0.1) #rgba(230,230,230,0.5)
                        ),
                      i, j)
        
        fig.add_trace(go.Bar(x=data_r["jour"], y=data_r[val],
                            marker=dict(color = data_r[val + "_new"], coloraxis="coloraxis"), ),
                      i, j)
        
        rangemin = "2020-03-15"
        if "dc" in val:
            rangemin = "2020-03-25"
        fig.update_xaxes(title_text="", range=[rangemin, "2020-05-10"], gridcolor='white', ticks="inside", tickformat='%d/%m', tickangle=0, nticks=10, linewidth=1, linecolor='white', row=i, col=j)
        fig.update_yaxes(title_text="", range=[0, max_value], gridcolor='white', linewidth=1, linecolor='white', row=i, col=j)

        j+=1
        if j == nj+1 or ((i >= 3) & (j == nj)):
            i+=1
            j=1
            
        """if j == nj+1:
            i+=1
            j=1"""

    for i in fig['layout']['annotations']:
        i['font'] = dict(size=15)

    #for annotation in fig['layout']['annotations']: 
            #annotation ['x'] = 0.5
    by_million_title = ""
    by_million_legend = ""
    if "pop" in val:
        by_million_title = "pour 1 million d'habitants, "
        by_million_legend = "pour 1M. d'hab."
    
    

    fig.update_layout(
        barmode="overlay",
        margin=dict(
            l=0,
            r=15,
            b=0,
            t=170,
            pad=0
        ),
        bargap=0,
        paper_bgcolor='#fffdf5',#fcf8ed #faf9ed
        plot_bgcolor='#f5f0e4',#f5f0e4 fcf8ed f0e8d5 
        coloraxis=dict(colorscale=["green", "#ffc832", "#cf0000"], cmin=-max_value_diff, cmax=max_value_diff), 
                    coloraxis_colorbar=dict(
                        title="Solde quotidien de<br>pers. {}<br>{}<br> &#8205; ".format(type_ppl, by_million_legend),
                        thicknessmode="pixels", thickness=15,
                        lenmode="pixels", len=400,
                        yanchor="bottom", y=0.26, xanchor="left", x=0.85,
                        ticks="outside", tickprefix="  ", ticksuffix=" hosp.",
                        nticks=15,
                        tickfont=dict(size=12),
                        titlefont=dict(size=15)),
                      
                    showlegend=False,
    
                     title={
                        'text': ("COVID19 : <b>nombre de personnes {}</b><br>"+sub).format(type_ppl, by_million_title),
                        'y':0.97,
                        'x':0.5,
                        'xref':"paper",
                         'yref':"container",
                        'xanchor': 'center',
                        'yanchor': 'middle'},
                        titlefont = dict(
                        size=35,
                        )
    )

    fig["layout"]["annotations"] += ( dict(
                            x=0.9,
                            y=0.015,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='Source :<br>Santé Publique France',
                            showarrow = False,
                            font=dict(size=12), 
                            opacity=0.5
                        ),
                            dict(
                            x=0.9,
                            y=0.21,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='25 % des données sont<br>comprises dans la courbe<br>grise la plus foncée,<br><br>50 % dans la deuxième,<br><br>75 % dans la troisième,<br><br>100 % dans la plus claire.',
                            showarrow = False,
                            font=dict(size=16), 
                            opacity=1,
                            align='left'
                        ))
    
    name_fig = "subplots_" + val 
    fig.write_image("images/charts/france/{}.png".format(name_fig), scale=2, width=1300, height=1600)

    fig["layout"]["annotations"] += (
                    dict(
                        x=0.5,
                        y=1,
                        xref='paper',
                        yref='paper',
                        xanchor='center',
                        text='Cliquez sur des éléments de légende pour les ajouter/supprimer',
                        showarrow = False
                        ),
                        )
    plotly.offline.plot(fig, filename = 'images/html_exports/france/{}.html'.format(name_fig), auto_open=False)
    print("> " + name_fig)


    #fig.show()

/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



> subplots_hosp_regpop
> subplots_rea_regpop


In [ ]:

dta = df_sursaud[df_sursaud["dep"] == "18"]
fig = go.Figure()
clrs = ["black"] * len(dta["taux_covid"])
fig.add_trace(go.Bar(x = dta["date_de_passage"], y = dta["taux_covid"]*100, marker_color=clrs))
#fig.update_xaxes(range=["2020-04-15", "2020-05-12"])
fig.show()

In [ ]:
dta

In [ ]:

for val in ["hosp_deppop"]: #, "hosp", "rea", "rea_pop"
    ni, nj = 12, 9
    i, j = 1, 1
    
    regions_ordered = df_region[df_region['jour'] == dates[-1]].sort_values(by=[val], ascending=False)["regionName"].values
    regions_ordered = list(dict.fromkeys(list(regions_ordered)))[:]
    
    #df_region[val+"_new"] = df_region[val].diff()
    max_value = 100
    
    #deps_ordered = df[df['jour'] == dates[-1]].sort_values(by=[val], ascending=False)["departmentName"].values
    #deps_ordered = list(dict.fromkeys(list(deps_ordered)))[:]
    deps_ordered = np.array(list(dict.fromkeys(list(df["departmentName"].values)))[:])
    deps_ordered_nb = np.array(list(dict.fromkeys(list(df["dep"].values)))[:])
    
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "2A", "200")
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "2B", "201")
    
    ind_deps = np.argsort(deps_ordered_nb.astype(int))
    
    deps_ordered_nb = deps_ordered_nb[ind_deps]
    deps_ordered = deps_ordered[ind_deps]
    
    deps_ordered_nb = deps_ordered_nb.astype(str)
    
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "200", "2A")
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "201", "2B")
    
    titles = []
    k=0
    for case in range(1, ni * nj - 1):
        if case in [80, 81, 89, 90, 98, 99]:
            titles += [""] 
        else:
            titles += ["<b>" + deps_ordered_nb[k] + "</b> - " + deps_ordered[k] + " ↓"]
            k+=1

    fig = make_subplots(rows=ni, cols=nj, shared_yaxes=True, subplot_titles= titles, vertical_spacing = 0.025, horizontal_spacing = 0.002)
    #&#8681;
    
    df_nonobj = df.select_dtypes(exclude=['object'])
    df_nonobj['jour'] = df['jour']
    
    vals_quantiles=[]
    for q in range(25, 125, 25):
        vals_quantiles.append(df_nonobj.groupby('jour').quantile(q=q/100).reset_index())
    
    type_ppl = "hospitalisées"
    if "rea" in val:
        type_ppl = "en réanimation"
    max_values_diff=[]
    
    clrs_dep = []
    for dep in tqdm(deps_ordered_nb):
        data_dep = df[df["departmentName"] == dep]
        
        data_dep[val + "_new"] = data_dep[val].diff()
        ordered_values = data_dep.sort_values(by=[val + "_new"], ascending=False)[val + "_new"]
        max_values_diff += [ordered_values.quantile(.90)]
                
        df_dep_temp = df[df["dep"] == dep]
        
        values_y = 100 * df_dep_temp["rea"]/lits_reas[lits_reas["nom_dpt"] == deps_ordered[list(deps_ordered_nb).index(dep)]]["LITS"].values[0]
        clrs=[]
        for tx in values_y.values:
            if tx < 60:
                clrs += ["green"]
            elif tx < 80:
                clrs += ["orange"]
            else:
                clrs += ["red"]
        
        mean_last7 = values_y[-7:].mean()
        if mean_last7 < 60:
            clrs_dep += ["green"]
        elif mean_last7 < 80:
            clrs_dep += ["orange"]
        else:
            clrs_dep += ["red"]
            
        fig.add_trace(go.Bar(x = df["jour"], y = values_y,
                            marker_color=clrs),
                      i, j)
        
        fig.update_xaxes(title_text="", range=["2020-04-25", "2020-05-12"], gridcolor='white', showgrid=False, ticks="inside", tickformat='%d/%m', tickfont=dict(size=7), tickangle=0, nticks=6, linewidth=0, linecolor='white', row=i, col=j)
        fig.update_yaxes(title_text="", range=[0, 150], gridcolor='white', linewidth=0, linecolor='white', tickfont=dict(size=7), nticks=8, row=i, col=j)

        j+=1
        if j == nj+1 or ((i >= 9) & (j >= nj-1)): 
            i+=1
            j=1

    cnt=0
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14, color = clrs_dep[cnt])
        #print(regions_ordered.index( df[df['departmentName'] == deps_ordered[cnt]]['regionName'].values[0] ))
        cnt+=1
        
    #for annotation in fig['layout']['annotations']: 
            #annotation ['x'] = 0.5
    by_million_title = ""
    by_million_legend = ""
    if "pop" in val:
        by_million_title = "pour 100 000 habitants, "
        by_million_legend = "pour 100k hab."
        
    max_value_diff = np.mean(max_values_diff) * 1.7
    fig.update_layout(
        barmode='overlay',
        margin=dict(
            l=0,
            r=15,
            b=0,
            t=160,
            pad=0
        ),
        bargap=0,
        paper_bgcolor='#fffdf5',#fcf8ed #faf9ed
        plot_bgcolor='#f5f0e4',#f5f0e4 fcf8ed f0e8d5
        coloraxis=dict(colorscale=["green", "#ffc832", "#cf0000"], cmin=-max_value_diff, cmax=max_value_diff), 
                    coloraxis_colorbar=dict(
                        title="Solde quotidien de<br>pers. {}<br>{}<br> &#8205; ".format(type_ppl, by_million_legend),
                        thicknessmode="pixels", thickness=15,
                        lenmode="pixels", len=350,
                        yanchor="bottom", y=0.15, xanchor="left", x=0.87,
                        ticks="outside", tickprefix="  ", ticksuffix=" hosp.",
                        nticks=5,
                        tickfont=dict(size=12),
                        titlefont=dict(size=15)),
                      
                    showlegend=False,
    
                     title={
                        'text': "Indicateur 2 : <b>saturation des réanimations par les patients Covid</b>",
                        'y':0.98,
                        'x':0.5,
                        'xref':"paper",
                        'yref':"container",
                        'xanchor': 'center',
                        'yanchor': 'middle'},
                        titlefont = dict(
                            size=50
                        )
    )

    fig["layout"]["annotations"] += ( 
                        dict(
                            x=0.9,
                            y=0.17,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='Rouge : > 80 %<br><br><br>Orange : 60 à 80 %<br><br><br>Vert : < 60 %',
                            showarrow = False,
                            font=dict(size=16), 
                            opacity=1,
                            align='left'
                        ),
                        dict(
                            x=0.9,
                            y=0.015,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='Source :<br>Santé Publique France',
                            showarrow = False,
                            font=dict(size=15), 
                            opacity=0.5
                        ),
                            
                            dict(
                            x=0.5,
                            y=1.03,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='middle',
                            text="par rapport au nb de lits en réa. avant l'épidémie - guillaumerozier.fr",
                            showarrow = False,
                            font=dict(size=30), 
                            opacity=1
                        ),
                                    )
    
    name_fig = "subplots_deconf_indic2"
    fig.write_image("images/charts/france/{}.png".format(name_fig), scale=3, width=1600, height=2300)

    fig["layout"]["annotations"] += (
                    dict(
                        x=0.5,
                        y=1,
                        xref='paper',
                        yref='paper',
                        xanchor='center',
                        text='Cliquez sur des éléments de légende pour les ajouter/supprimer',
                        showarrow = False
                        ),
                        )
    plotly.offline.plot(fig, filename = 'images/html_exports/france/{}.html'.format(name_fig), auto_open=False)
    print("> " + name_fig)


    #fig.show()

In [ ]:
"""
for val in ["hosp_deppop"]: #, "hosp", "rea", "rea_pop"
    ni, nj = 12, 9
    i, j = 1, 1
    
    regions_ordered = df_region[df_region['jour'] == dates[-1]].sort_values(by=[val], ascending=False)["regionName"].values
    regions_ordered = list(dict.fromkeys(list(regions_ordered)))[:]
    
    #df_region[val+"_new"] = df_region[val].diff()
    max_value = 100
    
    #deps_ordered = df[df['jour'] == dates[-1]].sort_values(by=[val], ascending=False)["departmentName"].values
    #deps_ordered = list(dict.fromkeys(list(deps_ordered)))[:]
    deps_ordered = np.array(list(dict.fromkeys(list(df["departmentName"].values)))[:])
    deps_ordered_nb = np.array(list(dict.fromkeys(list(df["dep"].values)))[:])
    
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "2A", "200")
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "2B", "201")
    
    ind_deps = np.argsort(deps_ordered_nb.astype(int))
    
    deps_ordered_nb = deps_ordered_nb[ind_deps]
    deps_ordered = deps_ordered[ind_deps]
    
    deps_ordered_nb = deps_ordered_nb.astype(str)
    
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "200", "2A")
    deps_ordered_nb = np.char.replace(deps_ordered_nb, "201", "2B")
    
    titles = []
    k=0
    for case in range(1, ni * nj - 1):
        if case in [80, 81, 89, 90, 98, 99]:
            titles += [""] 
        else:
            titles += ["<b>" + deps_ordered_nb[k] + "</b> - " + deps_ordered[k]]
            k+=1

    fig = make_subplots(rows=ni, cols=nj, shared_yaxes=True, subplot_titles= titles, vertical_spacing = 0.025, horizontal_spacing = 0.002)
    #&#8681;
    
    df_nonobj = df.select_dtypes(exclude=['object'])
    df_nonobj['jour'] = df['jour']
    
    vals_quantiles=[]
    for q in range(25, 125, 25):
        vals_quantiles.append(df_nonobj.groupby('jour').quantile(q=q/100).reset_index())
    
    type_ppl = "hospitalisées"
    if "rea" in val:
        type_ppl = "en réanimation"
    max_values_diff=[]
    
    clrs_dep = []
    for dep in tqdm(deps_ordered_nb):
        data_dep = df[df["departmentName"] == dep]
        
        data_dep[val + "_new"] = data_dep[val].diff()
        ordered_values = data_dep.sort_values(by=[val + "_new"], ascending=False)[val + "_new"]
        max_values_diff += [ordered_values.quantile(.90)]
                
        df_dep_temp = df[df["dep"] == dep]
        
        values_y = 100 * df_dep_temp["rea"]/lits_reas[lits_reas["nom_dpt"] == deps_ordered[list(deps_ordered_nb).index(dep)]]["LITS"].values[0]
        clrs=[]
        for tx in values_y.values:
            if tx < 60:
                clrs += ["green"]
            elif tx < 80:
                clrs += ["orange"]
            else:
                clrs += ["red"]
        
        mean_last7 = values_y[-7:].mean()
        if mean_last7 < 60:
            clrs_dep += ["green"]
        elif mean_last7 < 80:
            clrs_dep += ["orange"]
        else:
            clrs_dep += ["red"]
            
        #fig.add_trace(go.Bar(x = df["jour"], y = values_y,
                        #    marker_color=clrs),
                    #  i, j)
        fig.add_trace(go.Scatter(x = df[df["dep"] == dep]["jour"], y = df[df["dep"] == dep][""],
                            marker_color=clrs),
                      i, j)
        
        fig.update_xaxes(title_text="", range=["2020-04-25", "2020-05-12"], gridcolor='white', showgrid=False, ticks="inside", tickformat='%d/%m', tickfont=dict(size=7), tickangle=0, nticks=6, linewidth=0, linecolor='white', row=i, col=j)
        #fig.update_yaxes(title_text="", range=[0, 150], gridcolor='white', linewidth=0, linecolor='white', tickfont=dict(size=7), nticks=8, row=i, col=j)

        j+=1
        if j == nj+1 or ((i >= 9) & (j >= nj-1)): 
            i+=1
            j=1

    cnt=0
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14, color = clrs_dep[cnt])
        #print(regions_ordered.index( df[df['departmentName'] == deps_ordered[cnt]]['regionName'].values[0] ))
        cnt+=1
        
    #for annotation in fig['layout']['annotations']: 
            #annotation ['x'] = 0.5
    by_million_title = ""
    by_million_legend = ""
    if "pop" in val:
        by_million_title = "pour 100 000 habitants, "
        by_million_legend = "pour 100k hab."
        
    max_value_diff = np.mean(max_values_diff) * 1.7
    fig.update_layout(
        barmode='overlay',
        margin=dict(
            l=0,
            r=15,
            b=0,
            t=160,
            pad=0
        ),
        bargap=0,
        paper_bgcolor='#fffdf5',#fcf8ed #faf9ed
        plot_bgcolor='#f5f0e4',#f5f0e4 fcf8ed f0e8d5
        coloraxis=dict(colorscale=["green", "#ffc832", "#cf0000"], cmin=-max_value_diff, cmax=max_value_diff), 
                    coloraxis_colorbar=dict(
                        title="Solde quotidien de<br>pers. {}<br>{}<br> &#8205; ".format(type_ppl, by_million_legend),
                        thicknessmode="pixels", thickness=15,
                        lenmode="pixels", len=350,
                        yanchor="bottom", y=0.15, xanchor="left", x=0.87,
                        ticks="outside", tickprefix="  ", ticksuffix=" hosp.",
                        nticks=5,
                        tickfont=dict(size=12),
                        titlefont=dict(size=15)),
                      
                    showlegend=False,
    
                     title={
                        'text': "Indicateur 2 : <b>saturation des réanimations par les patients Covid</b>",
                        'y':0.98,
                        'x':0.5,
                        'xref':"paper",
                        'yref':"container",
                        'xanchor': 'center',
                        'yanchor': 'middle'},
                        titlefont = dict(
                            size=50
                        )
    )

    fig["layout"]["annotations"] += ( 
                        dict(
                            x=0.9,
                            y=0.17,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='Rouge : > 80 %<br><br><br>Orange : 60 à 80 %<br><br><br>Vert : < 60 %',
                            showarrow = False,
                            font=dict(size=16), 
                            opacity=1,
                            align='left'
                        ),
                        dict(
                            x=0.9,
                            y=0.015,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='top',
                            text='Source :<br>Santé Publique France',
                            showarrow = False,
                            font=dict(size=15), 
                            opacity=0.5
                        ),
                            
                            dict(
                            x=0.5,
                            y=1.03,
                            xref='paper',
                            yref='paper',
                            xanchor='center',
                            yanchor='middle',
                            text="par rapport au nb de lits en réa. avant l'épidémie - guillaumerozier.fr",
                            showarrow = False,
                            font=dict(size=30), 
                            opacity=1
                        ),
                                    )
    
    name_fig = "subplots_deconf_synthese"
    fig.write_image("images/charts/france/{}.png".format(name_fig), scale=3, width=1600, height=2300)

    fig["layout"]["annotations"] += (
                    dict(
                        x=0.5,
                        y=1,
                        xref='paper',
                        yref='paper',
                        xanchor='center',
                        text='Cliquez sur des éléments de légende pour les ajouter/supprimer',
                        showarrow = False
                        ),
                        )
    plotly.offline.plot(fig, filename = 'images/html_exports/france/{}.html'.format(name_fig), auto_open=False)
    print("> " + name_fig)


    #fig.show()"""

In [ ]:
df_dep_temp = df[df["dep"] == "01"]

#df_dep_temp["rea"]/lits_reas[lits_reas["nom_dpt"] == deps_ordered[ list(deps_ordered_nb).index("21")] ]["LITS"].values[0]
deps_ordered[ list(deps_ordered_nb).index("974")]
        